In [0]:
# ============================================================================
# Camada Gold - Métricas Performance Investimentos - Magic: The Gathering  
# Pipeline 100% PySpark DataFrame API
# ============================================================================

# ============================================================================
# BIBLIOTECAS UTILIZADAS
# ============================================================================
import logging
from datetime import datetime
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *
from pyspark.sql.types import *

# ============================================================================
# IMPORTAÇÃO DO MÓDULO UTILITÁRIO
# ============================================================================
%run ./gold_utils

# ============================================================================
# FUNÇÕES ESPECÍFICAS DE PERFORMANCE
# ============================================================================
def calcular_retorno(df, periodo_dias):
    """Calcula retorno percentual para um período específico"""
    window_periodo = Window.partitionBy("NME_CARD", "COD_SET").orderBy(col("prices.DT_INGESTION"))
    
    df_retorno = df.withColumn(f"VLR_USD_{periodo_dias}D_ANTES", 
                              lag("VLR_USD", periodo_dias).over(window_periodo))
    
    df_retorno = df_retorno.withColumn(f"RETORNO_{periodo_dias}D",
        when(col(f"VLR_USD_{periodo_dias}D_ANTES").isNotNull() & 
             (col(f"VLR_USD_{periodo_dias}D_ANTES") != 0),
             ((col("VLR_USD") - col(f"VLR_USD_{periodo_dias}D_ANTES")) / 
              col(f"VLR_USD_{periodo_dias}D_ANTES")).cast("float"))
        .otherwise(lit(None)))
    
    return df_retorno

def calcular_volatilidade(df, periodo_dias):
    """Calcula volatilidade (desvio padrão) para um período"""
    window_vol = Window.partitionBy("NME_CARD", "COD_SET") \
                      .orderBy(col("prices.DT_INGESTION")) \
                      .rowsBetween(-periodo_dias, 0)
    
    df_vol = df.withColumn(f"VOLATILIDADE_{periodo_dias}D",
                          stddev("VLR_USD").over(window_vol).cast("float"))
    
    return df_vol

def calcular_sharpe_ratio(df, periodo_dias, risk_free_rate=0.02):
    """Calcula Sharpe Ratio para um período"""
    daily_rf = risk_free_rate / 365
    
    df_sharpe = df.withColumn(f"SHARPE_RATIO_{periodo_dias}D",
        when(col(f"VOLATILIDADE_{periodo_dias}D").isNotNull() & 
             (col(f"VOLATILIDADE_{periodo_dias}D") != 0),
             ((col(f"RETORNO_{periodo_dias}D") - daily_rf) / 
              col(f"VOLATILIDADE_{periodo_dias}D")).cast("float"))
        .otherwise(lit(None)))
    
    return df_sharpe

# Inicializar processador com configuração manual 
def get_secret(secret_name, default_value=None):
    try:
        return dbutils.secrets.get(scope="mtg-pipeline", key=secret_name)
    except:
        if default_value is not None:
            print(f"Segredo '{secret_name}' não encontrado, usando valor padrão")
            return default_value
        else:
            print(f"Segredo obrigatório '{secret_name}' não encontrado")
            raise Exception(f"Segredo '{secret_name}' não configurado")


# ============================================================================
# CONFIGURAÇÃO COM MÓDULO UTILITÁRIO
# ============================================================================
TABLE_NAME = "TB_METRICAS_PERFORMANCE_INVESTIMENTOS"

# Inicializar processador com configuração manual 
config = create_manual_config(
    catalog_name="magic_the_gathering", 
    s3_bucket=get_secret("s3_bucket"),  
    s3_gold_prefix="magic_the_gathering/gold"
)
processor = GoldTableProcessor(TABLE_NAME, config)
spark = processor.spark

In [0]:
# ============================================================================
# CARREGAMENTO DE DADOS SILVER COM MÓDULO
# ============================================================================
dfs = processor.load_silver_data(['cards', 'prices'])
df_cards = dfs['cards']
df_prices = dfs['prices']

# ============================================================================
# JOIN PRINCIPAL E RESOLUÇÃO DE AMBIGUIDADES
# ============================================================================
df_base = df_cards.join(df_prices, ["NME_CARD", "COD_SET"], "inner")


In [0]:
# ============================================================================
# FILTRAR DADOS COM HISTÓRICO SUFICIENTE
# ============================================================================
window_count = Window.partitionBy("NME_CARD", "COD_SET")
df_base = df_base.withColumn("QTD_REGISTROS_CARTA", count("*").over(window_count)) \
                 .filter(col("QTD_REGISTROS_CARTA") >= 30)

# ============================================================================
# CÁLCULOS DE PERFORMANCE POR PERÍODO
# ============================================================================
df_performance = df_base

# Calcular retornos para diferentes períodos
for periodo in [7, 30, 90, 365]:
    df_performance = calcular_retorno(df_performance, periodo)
    df_performance = calcular_volatilidade(df_performance, periodo)
    df_performance = calcular_sharpe_ratio(df_performance, periodo)

# ============================================================================
# CÁLCULO DE MAXIMUM DRAWDOWN
# ============================================================================
window_max = Window.partitionBy("NME_CARD", "COD_SET").orderBy(col("prices.DT_INGESTION")) \
                  .rowsBetween(Window.unboundedPreceding, 0)

df_performance = df_performance.withColumn("PRECO_MAXIMO_HISTORICO", 
                                         max("VLR_USD").over(window_max))

df_performance = df_performance.withColumn("DRAWDOWN_ATUAL",
    (col("VLR_USD") - col("PRECO_MAXIMO_HISTORICO")) / col("PRECO_MAXIMO_HISTORICO"))

# Maior drawdown nos últimos 90 dias
window_90d = Window.partitionBy("NME_CARD", "COD_SET") \
                  .orderBy(col("prices.DT_INGESTION")) \
                  .rowsBetween(-90, 0)

df_performance = df_performance.withColumn("MAX_DRAWDOWN_90D",
                                         min("DRAWDOWN_ATUAL").over(window_90d).cast("float"))

# ============================================================================
# CLASSIFICAÇÃO DE RISCO E SCORING
# ============================================================================
df_performance = df_performance.withColumn("PERFIL_RISCO",
    when(col("VOLATILIDADE_30D") < 0.1, "Conservador")
    .when(col("VOLATILIDADE_30D") < 0.2, "Moderado")
    .otherwise("Agressivo"))

# Scores normalizados (0-100)
df_performance = df_performance.withColumn("SCORE_RETORNO_30D",
    when(col("RETORNO_30D").isNotNull(),
         greatest(least(col("RETORNO_30D") * 100 + 50, lit(100)), lit(0)))
    .otherwise(lit(50)))

df_performance = df_performance.withColumn("SCORE_SHARPE_30D",
    when(col("SHARPE_RATIO_30D").isNotNull(),
         greatest(least(col("SHARPE_RATIO_30D") * 20 + 50, lit(100)), lit(0)))
    .otherwise(lit(50)))

df_performance = df_performance.withColumn("SCORE_DRAWDOWN",
    when(col("MAX_DRAWDOWN_90D").isNotNull(),
         greatest(least(-col("MAX_DRAWDOWN_90D") * 100 + 50, lit(100)), lit(0)))
    .otherwise(lit(50)))

# Investment Score final (0-100)
df_performance = df_performance.withColumn("INVESTMENT_SCORE",
    round((col("SCORE_RETORNO_30D") * 0.4 + 
           col("SCORE_SHARPE_30D") * 0.3 + 
           col("SCORE_DRAWDOWN") * 0.3), 2))



In [0]:
# ============================================================================
# AGREGAÇÃO FINAL POR CARTA
# ============================================================================
# Selecionar colunas explicitamente para evitar ambiguidade
df_performance_clean = df_performance.select(
    "NME_CARD", "COD_SET", "NME_SET", "NME_CARD_TYPE",
    col("cards.NME_RARITY").alias("NME_RARITY"), "NME_ARTIST",
    "VLR_USD", "PRECO_MAXIMO_HISTORICO", 
    "RETORNO_7D", "RETORNO_30D", "RETORNO_90D", "RETORNO_365D",
    "VOLATILIDADE_7D", "VOLATILIDADE_30D", "VOLATILIDADE_90D", "VOLATILIDADE_365D",
    "SHARPE_RATIO_7D", "SHARPE_RATIO_30D", "SHARPE_RATIO_90D", "SHARPE_RATIO_365D",
    "MAX_DRAWDOWN_90D", "PERFIL_RISCO", "INVESTMENT_SCORE",
    col("prices.DT_INGESTION")
)

df_final = df_performance_clean.groupBy(
    "NME_CARD", "COD_SET", "NME_SET", "NME_CARD_TYPE",
    "NME_RARITY", "NME_ARTIST"
).agg(
    # Dados atuais
    max("DT_INGESTION").alias("DT_ULTIMA_ATUALIZACAO"),
    last("VLR_USD").cast("float").alias("PRECO_ATUAL"),
    last("PRECO_MAXIMO_HISTORICO").cast("float").alias("PRECO_MAXIMO_HISTORICO"),
    
    # Performance por período
    last("RETORNO_7D").cast("float").alias("RETORNO_7D"),
    last("RETORNO_30D").cast("float").alias("RETORNO_30D"),
    last("RETORNO_90D").cast("float").alias("RETORNO_90D"),
    last("RETORNO_365D").cast("float").alias("RETORNO_365D"),
    
    # Volatilidade por período
    last("VOLATILIDADE_7D").cast("float").alias("VOLATILIDADE_7D"),
    last("VOLATILIDADE_30D").cast("float").alias("VOLATILIDADE_30D"),
    last("VOLATILIDADE_90D").cast("float").alias("VOLATILIDADE_90D"),
    last("VOLATILIDADE_365D").cast("float").alias("VOLATILIDADE_365D"),
    
    # Sharpe Ratio por período
    last("SHARPE_RATIO_7D").cast("float").alias("SHARPE_RATIO_7D"),
    last("SHARPE_RATIO_30D").cast("float").alias("SHARPE_RATIO_30D"),
    last("SHARPE_RATIO_90D").cast("float").alias("SHARPE_RATIO_90D"),
    last("SHARPE_RATIO_365D").cast("float").alias("SHARPE_RATIO_365D"),
    
    # Drawdown e Risco
    last("MAX_DRAWDOWN_90D").cast("float").alias("MAX_DRAWDOWN_90D"),
    last("PERFIL_RISCO").alias("PERFIL_RISCO"),
    last("INVESTMENT_SCORE").cast("float").alias("INVESTMENT_SCORE"),
    
    # Métricas agregadas
    avg("VLR_USD").cast("float").alias("PRECO_MEDIO_HISTORICO"),
    count("*").cast("int").alias("QTD_DIAS_HISTORICO")
)

# ============================================================================
# ADICIONAR DATA DE REFERÊNCIA E RANKINGS
# ============================================================================
df_final = df_final.withColumn("DATA_REF", current_date())

# Rankings por diferentes critérios
window_rank_retorno = Window.orderBy(col("RETORNO_30D").desc())
window_rank_sharpe = Window.orderBy(col("SHARPE_RATIO_30D").desc())
window_rank_score = Window.orderBy(col("INVESTMENT_SCORE").desc())

df_final = df_final.withColumn("RANK_RETORNO_30D", row_number().over(window_rank_retorno).cast("int")) \
                   .withColumn("RANK_SHARPE_30D", row_number().over(window_rank_sharpe).cast("int")) \
                   .withColumn("RANK_INVESTMENT_SCORE", row_number().over(window_rank_score).cast("int"))

# ============================================================================
# ESCRITA DA TABELA COM MÓDULO
# ============================================================================
processor.save_gold_table(df_final, partition_cols=["NME_SET"])

# ============================================================================
# FINALIZAÇÃO
# ============================================================================
print("TB_METRICAS_PERFORMANCE_INVESTIMENTOS modularizada criada com sucesso!")
